In [1]:
import numpy as np
import duckdb
from vicinity import Vicinity, Backend, Metric
from vicinity.datatypes import QueryResult
import json
import pandas as pd
from numpy import typing as npt
from typing import Optional, Iterable

import numpy as np

from utils import numpy_similarity_calculation, numpy_similarity_threshold_calculation, build_where_clause

In [10]:
def build_where_clause(filters: dict) -> str:
    """
    Convert a dictionary of filters to a DuckDB WHERE clause.
    
    Supports operators:
    - eq: equals (default when no operator specified)
    - gt: greater than
    - gte: greater than or equal
    - lt: less than
    - lte: less than or equal
    - in: list membership
    - between: range inclusive
    
    Example:
    filters = {
        'type': 'fire',                    # equals
        'attack__gt': 80,                  # greater than
        'defense__lte': 100,               # less than or equal
        'name__in': ['charizard', 'moltres'], # in list
        'speed__between': [50, 100]        # between range
    }
    """
    operators = {
        'gt': '>',
        'gte': '>=',
        'lt': '<',
        'lte': '<=',
        'in': 'IN',
        'between': 'BETWEEN'
    }
    
    conditions = []
    
    for key, value in filters.items():
        parts = key.split('__')
        field = parts[0]
        op = parts[1] if len(parts) > 1 else 'eq'
        
        if op == 'eq':
            conditions.append(f"{field} = '{value}'" if isinstance(value, str) else f"{field} = {value}")
        elif op == 'in':
            formatted_values = [f"'{v}'" if isinstance(v, str) else str(v) for v in value]
            conditions.append(f"{field} IN ({', '.join(formatted_values)})")
        elif op == 'between':
            conditions.append(f"{field} BETWEEN {value[0]} AND {value[1]}")
        elif op in operators:
            value_str = f"'{value}'" if isinstance(value, str) else str(value)
            conditions.append(f"{field} {operators[op]} {value_str}")
            
    return ' AND '.join(conditions) if conditions else '1=1'

In [7]:
conn = duckdb.connect(":memory:")
conn.sql("INSTALL json; LOAD json")

In [8]:
data = conn.sql("""
    SELECT ROW_NUMBER() OVER () - 1 as _idx, * FROM (
    SELECT UNNEST(items, recursive:= true) 
    AS items from 'test_data.json')
""")

# data = conn.sql("""
#     SELECT ROW_NUMBER() OVER () - 1 as index, *
#     FROM data
# """)

# data = conn.sql("""
#     SELECT item as items
#     FROM (
#         SELECT UNNEST(items) WITH ORDINALITY AS (item, ord)
#         FROM 'test_data.json'
#     )
# """)

In [11]:
filters = {
    # 'type': 'fire',
    # 'attack__gt': 80,
    # 'defense__lte': 100,
    'name__in': ['charizard', 'moltres'],
    # 'speed__between': [50, 100]
}
where_clause = build_where_clause(filters)
rel = conn.sql(f"""
    SELECT *
    FROM data
    WHERE {where_clause}
""")
items = rel.df().to_dict(orient='records')

In [12]:
items

[{'_idx': 25,
  'name': 'charizard',
  'type': 'fire',
  'attack': 84,
  'defense': 78,
  'speed': 100},
 {'_idx': 15,
  'name': 'moltres',
  'type': 'fire',
  'attack': 100,
  'defense': 90,
  'speed': 90}]

In [14]:
indices = rel.df()['_idx'].tolist()
indices

[25, 15]

In [7]:
with open('test_data.json', 'r') as f:
    items = json.load(f)['items']
    items = [{"_idx": i, **item} for i, item in enumerate(items)]
    json_str = json.dumps(items)
    df = pd.DataFrame(items)

df.head()

,_idx,name,type,attack,defense,speed
0,0,bulbasaur,grass,49,49,45
1,1,charmander,fire,52,43,65
2,2,squirtle,water,48,65,43
3,3,pikachu,electric,55,40,90
4,4,jigglypuff,normal,20,45,20


In [8]:
class Test:
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def query(self, filters: dict) -> pd.DataFrame:
        where_clause = build_where_clause(filters)
        return duckdb.query(f"""
            SELECT *
            FROM self.df
            WHERE {where_clause}
        """).df()

filters = {
    # 'type': 'fire',
    # 'attack__gt': 80,
    # 'defense__lte': 100,
    'name__in': ['charizard', 'moltres', 'poop'],
    # 'speed__between': [50, 100]
}
# where_clause = build_where_clause(filters)
# duckdb.query(f"""
#     SELECT *
#     FROM df
#     WHERE {where_clause}
# """).df().head()

test = Test(df)
test.query(filters)

,_idx,name,type,attack,defense,speed
0,25,charizard,fire,84,78,100
1,15,moltres,fire,100,90,90


In [86]:
with open('test_data.json', 'r') as f:
    items = json.load(f)['items']
    items = [{"_idx": i, **item} for i, item in enumerate(items)]
    json_str = json.dumps(items)
    df = pd.DataFrame(items)

# data = conn.sql("""
#     SELECT * FROM df
# """)

record = {
    "name": "poop",
    "type": "fire",
    "attack": 100,
    "defense": 100,
    "speed": 100
}
conn.sql("DROP TABLE IF EXISTS data")
conn.sql("CREATE TABLE data AS SELECT * FROM df")

# data = conn.sql(f"""
#     INSERT INTO data 
#     SELECT 
#         (SELECT MAX(_idx) + 1 FROM data),
#         '{record['name']}',
#         '{record['type']}',
#         {record['attack']},
#         {record['defense']},
#         {record['speed']}
# """)

data = conn.sql(f"""
    INSERT INTO data 
    SELECT 
        (SELECT MAX(_idx) + 1 FROM data),
        {', '.join(f"'{v}'" if isinstance(v, str) else str(v) for v in record.values())}
""")

filters = {
    # 'type': 'fire',
    # 'attack__gt': 80,
    # 'defense__lte': 100,
    'name__in': ['charizard', 'moltres', 'poop'],
    # 'speed__between': [50, 100]
}
where_clause = build_where_clause(filters)
rel = conn.sql(f"""
    SELECT *
    FROM data
    WHERE {where_clause}
""")

indices = rel.df()['_idx'].tolist()
indices


[25, 15, 30]

In [67]:
items

[{'_idx': 25,
  'name': 'charizard',
  'type': 'fire',
  'attack': 84,
  'defense': 78,
  'speed': 100},
 {'_idx': 15,
  'name': 'moltres',
  'type': 'fire',
  'attack': 100,
  'defense': 90,
  'speed': 90}]

In [21]:
# Create some dummy data
# read from file
with open('test_data.json', 'r') as f:
    data = json.load(f)

# pokemen dicts with name, type, attack, defense, speed
items = data['items']
vectors: list[np.array] = [np.random.rand(128) for _ in range(len(items))]

df = pd.DataFrame(
    {
        "items": items,
        "vectors": vectors
    }
)
df.head()

,items,vectors
0,"{'name': 'bulbasaur', 'type': 'grass', 'attack...","[0.28396959732666105, 0.6192961206329993, 0.84..."
1,"{'name': 'charmander', 'type': 'fire', 'attack...","[0.19322161205903732, 0.8449598748932119, 0.63..."
2,"{'name': 'squirtle', 'type': 'water', 'attack'...","[0.5799893256300817, 0.4039190347768197, 0.742..."
3,"{'name': 'pikachu', 'type': 'electric', 'attac...","[0.697850137586721, 0.20588236212847, 0.901883..."
4,"{'name': 'jigglypuff', 'type': 'normal', 'atta...","[0.12160324372656617, 0.5526122433120781, 0.15..."


In [77]:
df.iloc[0]

_idx               0
name       bulbasaur
type           grass
attack            49
defense           49
speed             45
Name: 0, dtype: object

In [33]:
df.loc[15]

items      {'name': 'moltres', 'type': 'fire', 'attack': ...
vectors    [0.040648006287507754, 0.5106881240073904, 0.7...
Name: 15, dtype: object

In [34]:
df.loc[25]


items      {'name': 'charizard', 'type': 'fire', 'attack'...
vectors    [0.37502634514324396, 0.7928920353366223, 0.13...
Name: 25, dtype: object

In [30]:


# Initialize the Vicinity instance (using the basic backend and cosine metric)
vicinity = Vicinity.from_vectors_and_items(
    vectors=np.array(vectors),
    items=items,
    backend_type=Backend.BASIC,
    metric=Metric.COSINE,
    store_vectors=True,
)

In [31]:
vicinity.get_vector_by_index([25, 15])

array([[3.75026345e-01, 7.92892035e-01, 1.32182338e-01, 6.04819474e-01,
        9.64939285e-01, 5.33784853e-01, 7.90466825e-01, 1.15128043e-01,
        6.45050162e-01, 8.76488991e-01, 5.92628405e-01, 7.43047612e-01,
        9.92675257e-01, 5.83451462e-01, 6.38270335e-01, 4.88791494e-01,
        5.88041917e-01, 7.92462474e-02, 2.77439313e-01, 5.23783865e-01,
        3.12560717e-01, 6.85118047e-01, 6.85129384e-01, 9.90944506e-01,
        8.16328082e-01, 2.45108737e-01, 1.04350520e-01, 5.42290666e-01,
        8.93767060e-01, 7.26711604e-01, 3.34674289e-01, 9.00808335e-01,
        8.61078641e-01, 1.21486204e-01, 6.60600663e-01, 7.98997566e-01,
        3.93576187e-01, 5.07899834e-01, 4.50928542e-01, 4.00005747e-01,
        2.01335623e-01, 8.01228189e-01, 4.76205720e-01, 2.56854696e-02,
        4.07864025e-01, 5.60736612e-01, 2.07360005e-01, 6.81261412e-01,
        5.03323809e-01, 9.97281954e-02, 8.21711292e-01, 1.55250059e-02,
        3.57412753e-02, 3.98621161e-01, 5.30701195e-01, 6.384015

In [59]:
from dataclasses import dataclass

@dataclass
class QueryMethod:
    name: str
    args: dict
    kwargs: dict


class VectorDB:
    def __init__(self, 
                 name: str, 
                 backend: Backend, 
                 embedder_fn: callable,
                 embed_on: Optional[str] = None,
                 metric: Optional[Metric] = Metric.COSINE,
                 items: Optional[list[dict | str]] = None,
                 vectors: Optional[list[np.array]] = None,
                 ):
        self.name = name
        self.backend = backend
        self.metric = metric
        self.embedder_fn = embedder_fn
        self.embed_on = embed_on
        self.store: Vicinity | None = self._create_store(items, vectors)
        self.methods: list[QueryMethod] = list()
        self.df: pd.DataFrame | None = self._create_df(items)

    @property
    def items(self) -> list[dict | str] | None:
        if self.store is None:
            raise ValueError("Store is not initialized")
        return self.store.items
    
    def _create_df(self, items: Optional[list[dict | str]] = None) -> pd.DataFrame | None:
        if items is None:
            return None
        items = [{"_idx": i, **item} for i, item in enumerate(items)]
        return pd.DataFrame(items)
    
    def _update_df(self, items_to_add: list[dict | str]) -> None:
        if self.df is None:
            self.df = self._create_df(items_to_add)
        else:
            # TODO: this is inefficient, but it works for now
            items = self.df['items'].tolist()
            items.extend(items_to_add)
            items = [{"_idx": i, **item} for i, item in enumerate(items)]
            self.df = pd.DataFrame(items)
    
    def _create_store(self, items: Optional[list[dict | str]] = None, vectors: Optional[list[np.array]] = None) -> Vicinity | None:
        kwargs = {
            'backend_type': self.backend,
            'store_vectors': True,
        }
        if self.metric is not None:
            kwargs['metric'] = self.metric
        if items is not None and vectors is not None:
            return Vicinity.from_vectors_and_items(
                vectors=np.array(vectors),
                items=items,
                **kwargs,
            )
        elif items is not None:
            vectors: npt.NDArray  = self.embedder_fn(items)
            return Vicinity.from_vectors_and_items(
                vectors=vectors,
                items=items,
                **kwargs,
            )
        else:
            return None
    
    def insert(self, items: list[dict | str], embed_on: Optional[str] = None) -> None:
        vectors: npt.NDArray  = self.embedder_fn(items)
        if isinstance(items[0], str):
            vectors = self.embedder_fn(items)
        else:
            if embed_on is None and self.embed_on is None:
                raise ValueError("embed_on field must be provided if items is a list of dicts")
            vectors = self.embedder_fn([item[embed_on] for item in items])
        if self.store is None:
            self.store = self._create_store(items, vectors)
            
        else:
            self.store.insert(items, vectors)

    def delete(self, items: list[dict | str]) -> None:
        if self.store is None:
            raise ValueError("Store is not initialized")
        self.store.delete(items)

    def delete_by_index(self, indices: list[int]) -> None:
        if self.store is None:
            raise ValueError("Store is not initialized")
        self.store.backend.delete(indices)
        
    def _filter_and_get_indices(self, filters: dict) -> list[int]:
        if self.store is None:
            raise ValueError("Store is not initialized")
        where_clause = build_where_clause(filters)
        return duckdb.query(f"""
            SELECT _idx
            FROM self.df
            WHERE {where_clause}
        """).df()['_idx'].tolist()
    
    def _similarity_and_get_indices(self, text: str, k: int = 10) -> list[list[int, float]]:
        query_vector = self.embedder_fn([text])[0]  # Get first vector since we only have one text
        query_results: list[QueryResult] = self.store.query(query_vector, k=k)
        indices = [result[0]for result in query_results[0]]
        scores = [result[1] for result in query_results[0]]
        return indices, scores
    
    def _similarity_threshold_and_get_indices(self, text: str, threshold: float) -> list[list[int, float]]:
        query_vector = self.embedder_fn([text])[0]  # Get first vector since we only have one text
        query_results: list[QueryResult] = self.store.query_threshold(query_vector, threshold=threshold)
        indices = [result[0] for result in query_results[0]]
        scores = [result[1] for result in query_results[0]]
        return indices, scores
    
    def filter(self, filters: dict) -> list[dict | str]:
        indices = self._filter_and_get_indices(filters)
        return [self.items[i] for i in indices]
    
    def similarity(self, text: str, k: int = 10, filters: Optional[dict] = None) -> list[tuple[dict | str, float]]:
        query_vector = self.embedder_fn([text])[0]
        if filters is None:
            results: list[QueryResult] = self.store.query(query_vector, k=k)
            return results[0]
        else:
            # First get filtered indices
            filtered_indices = set(self._filter_and_get_indices(filters))
            results: list[QueryResult] = self.store.query(query_vector, k=len(self.items))
            items_scores = [item_score for item_score in results[0] if item_score[0]['_idx'] in filtered_indices]
            return items_scores[:k]
    
    def similarity_threshold(self, text: str, threshold: float, max_k: Optional[int] = None, filters: Optional[dict] = None) -> list[tuple[dict | str, float]]:
        if filters is None:
            indices, scores = self._similarity_threshold_and_get_indices(text, threshold)
            result_pairs = list(zip([self.items[i] for i in indices], scores))
        else:
            # First get filtered indices
            filtered_indices = set(self._filter_and_get_indices(filters))
            # Get similarity scores for items above threshold
            indices, scores = self._similarity_threshold_and_get_indices(text, threshold)
            # Filter and create result pairs
            result_pairs = [(self.items[i], scores[idx]) 
                        for idx, i in enumerate(indices) 
                        if i in filtered_indices]
        
        return result_pairs[:max_k] if max_k is not None else result_pairs
    
 




In [55]:
def test_embedder(x: list[dict | str]) -> npt.NDArray:
    return np.array(
        [np.random.rand(128) for _ in range(len(x))]
    )


In [60]:
items

[{'name': 'bulbasaur',
  'type': 'grass',
  'attack': 49,
  'defense': 49,
  'speed': 45},
 {'name': 'charmander',
  'type': 'fire',
  'attack': 52,
  'defense': 43,
  'speed': 65},
 {'name': 'squirtle',
  'type': 'water',
  'attack': 48,
  'defense': 65,
  'speed': 43},
 {'name': 'pikachu',
  'type': 'electric',
  'attack': 55,
  'defense': 40,
  'speed': 90},
 {'name': 'jigglypuff',
  'type': 'normal',
  'attack': 20,
  'defense': 45,
  'speed': 20},
 {'name': 'meowth',
  'type': 'normal',
  'attack': 40,
  'defense': 35,
  'speed': 90},
 {'name': 'gyarados',
  'type': 'water',
  'attack': 125,
  'defense': 79,
  'speed': 81},
 {'name': 'dragonite',
  'type': 'dragon',
  'attack': 134,
  'defense': 95,
  'speed': 80},
 {'name': 'mewtwo',
  'type': 'psychic',
  'attack': 150,
  'defense': 90,
  'speed': 130},
 {'name': 'snorlax',
  'type': 'normal',
  'attack': 110,
  'defense': 65,
  'speed': 30},
 {'name': 'vaporeon',
  'type': 'water',
  'attack': 65,
  'defense': 60,
  'speed': 65

In [61]:
with open('test_data.json', 'r') as f:
    data = json.load(f)
    items = data['items']

# pokemen dicts with name, type, attack, defense, speed
items = data['items']
v = VectorDB(name='test', backend=Backend.BASIC, embedder_fn=lambda x: test_embedder(x))
v.insert(items, embed_on='name')
v.store.get_vector_by_index([0])

array([[2.66483201e-01, 9.70624630e-01, 6.93625408e-01, 4.33736747e-01,
        5.99337581e-01, 5.05066025e-01, 3.14909434e-01, 1.57292728e-01,
        5.69835546e-01, 1.12648845e-01, 5.78655072e-01, 2.41462457e-01,
        3.32366307e-01, 8.66081524e-01, 7.66321115e-01, 6.07893932e-01,
        4.78410666e-01, 4.63450715e-01, 5.46447309e-01, 6.09436451e-02,
        5.27137216e-01, 6.71980021e-01, 4.36729098e-01, 6.24618812e-01,
        8.62914948e-01, 2.35509709e-01, 7.95149790e-01, 7.20546032e-01,
        2.67751563e-01, 9.82932232e-02, 7.23425644e-01, 6.00268758e-01,
        9.06324017e-01, 3.27903846e-01, 5.76889000e-01, 2.04987815e-02,
        6.12845100e-01, 2.73267889e-01, 4.25987288e-01, 4.15699039e-01,
        2.09679341e-01, 1.60006285e-03, 6.78941679e-01, 9.59028710e-01,
        2.02623072e-01, 7.76544493e-01, 3.96989109e-01, 2.61943220e-01,
        2.66405245e-01, 3.78273633e-01, 6.14005785e-02, 9.20520517e-01,
        7.01964459e-01, 8.83472162e-01, 9.83493492e-01, 8.009365

In [66]:
v.similarity('bulbasaur', k=3, filters={'type': 'psychic'})


KeyError: '_idx'

In [ ]:
# Create a query vector
query_vector = np.random.rand(128)

# Query for nearest neighbors with a top-k search
results = vicinity.query(query_vector, k=3)

# Query for nearest neighbors with a threshold search
results = vicinity.query_threshold(query_vector, threshold=0.9)

# Query with a list of query vectors
query_vectors = np.random.rand(5, 128)
results = vicinity.query(query_vectors, k=3)

In [23]:
results

[[({'name': 'jigglypuff',
    'type': 'normal',
    'attack': 20,
    'defense': 45,
    'speed': 20},
   np.float64(0.2011369367132435)),
  ({'name': 'squirtle',
    'type': 'water',
    'attack': 48,
    'defense': 65,
    'speed': 43},
   np.float64(0.20559490384959989)),
  ({'name': 'alakazam',
    'type': 'psychic',
    'attack': 50,
    'defense': 45,
    'speed': 120},
   np.float64(0.20729572416868625))],
 [({'name': 'squirtle',
    'type': 'water',
    'attack': 48,
    'defense': 65,
    'speed': 43},
   np.float64(0.1952706383522812)),
  ({'name': 'venusaur',
    'type': 'grass',
    'attack': 82,
    'defense': 83,
    'speed': 80},
   np.float64(0.20359333789039336)),
  ({'name': 'moltres',
    'type': 'fire',
    'attack': 100,
    'defense': 90,
    'speed': 90},
   np.float64(0.21042273884180518))],
 [({'name': 'dragonite',
    'type': 'dragon',
    'attack': 134,
    'defense': 95,
    'speed': 80},
   np.float64(0.18262600126192852)),
  ({'name': 'charizard',
    'typ

In [17]:
vicinity.save('my_vector_store', overwrite=True)


In [93]:
items = ["triforce", "master sword", "hylian shield", "boomerang", "hookshot"]
vectors = np.random.rand(len(items), 128)

# Initialize the Vicinity instance (using the basic backend and cosine metric)
vicinity = Vicinity.from_vectors_and_items(
    vectors=vectors,
    items=items,
    backend_type=Backend.BASIC,
    metric=Metric.COSINE
)

# Create a query vector
query_vector = np.random.rand(128)

# Query for nearest neighbors with a top-k search
results = vicinity.query(query_vector, k=3)

# Query for nearest neighbors with a threshold search
results = vicinity.query_threshold(query_vector, threshold=0.9)

# Query with a list of query vectors
query_vectors = np.random.rand(5, 128)
results = vicinity.query(query_vectors, k=3)

In [94]:
results

[[('master sword', np.float64(0.21001252989131447)),
  ('hookshot', np.float64(0.2238990124026693)),
  ('hylian shield', np.float64(0.2342791889325595))],
 [('hookshot', np.float64(0.2288484498321841)),
  ('boomerang', np.float64(0.25615091406534374)),
  ('master sword', np.float64(0.2643195489060941))],
 [('boomerang', np.float64(0.2505118029186939)),
  ('triforce', np.float64(0.25388766981823396)),
  ('hookshot', np.float64(0.26849438096668377))],
 [('hylian shield', np.float64(0.23820765944967714)),
  ('hookshot', np.float64(0.25088416029164395)),
  ('master sword', np.float64(0.25218301583590397))],
 [('triforce', np.float64(0.2186518194708451)),
  ('hylian shield', np.float64(0.23307169100358138)),
  ('master sword', np.float64(0.24290509082314804))]]